In [1]:
import pickle
import networkx as nx
import pandas as pd
import herbSyner_Finder
from herbSyner_Finder import Herb_Comb_Community
from herbSyner_Finder import Detection_key_Module_multiple
from herbSyner_Finder import herb_synergy_landscape


Shrinking network to its LCC 15970 217160
Final shape: 15911 217109


c:\Users\yin\OneDrive\Project\project_group\manuscipt-jiaqi\pytomedicine\revise_v1\final\code\code\herbSyner_Finder\cal_network_distance\disease.py:17: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(self.disease_file_name)


In [2]:
from herbSyner_Finder import generate_objects

In [3]:
g_obj, ingredients_obj, herb_obj, herb_info, fangji, disease_obj, herb_distance_obj = generate_objects.load_obj_data()

Shrinking network to its LCC 15970 217160
Final shape: 15911 217109


c:\Users\yin\OneDrive\Project\project_group\manuscipt-jiaqi\pytomedicine\revise_v1\final\code\code\herbSyner_Finder\cal_network_distance\disease.py:17: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(self.disease_file_name)


# 1.  calculate network distance

In [4]:
# save file path 

save_path = "result"

HIf = 'result/herb_ingredient_pairs_pd.csv' # herb_ingre_pairs
HHf = 'result/herb_herb_dis_pd.csv' # herb_herb_dis
IIf = 'result/ingre_ingre_dis_pd.csv' # ingre_ingre_dis
DHf = 'result/herb_disease_pd.csv' # herb_disease_pd
DIf = 'result/herb_disease_ingre_pd.csv' # herb_disease_ingre
disease_name = "Cough Variant Asthma"
disease_list = ['Cough Variant Asthma']


## this step may take time longer time , with random is set to 100-1000 to get significance 

In [5]:
# %%time
# # use herb pair 'MA HUANG','ZI SU YE' as example

# combination_herb_list=['MA HUANG','ZI SU YE']

# herb_list, herb_herb_dis, ingre_ingre_dis, herb_ingre_pairs = herb_synergy_landscape.cal_combination_distance(herb_info,combination_herb_list)
# # here we use random time =  1 as example to save time
# herb_disease_pd, herb_disease_ingre_pd = herb_synergy_landscape.cal_combination_disease(herb_list, herb_distance_obj, disease_list, random_time=1)
    

In [6]:
# # save distance data from example as files
# herb_herb_dis.to_csv('result/herb_herb_distance_example.csv', index=None) 
# ingre_ingre_dis.to_csv('result/ingre_ingre_distance_example.csv', index=None)
# herb_ingre_pairs.to_csv('result/herb_ingredient_pairs_example.csv', index=None)
# herb_disease_pd.to_csv('result/herb_disease_pd_example.csv', index=None)
# herb_disease_ingre_pd.to_csv('result/herb_disease_ingre_pd_example.csv', index=None)

# 2. calculate the commnunities for the complex landscape between herbs

In [7]:
# reload data generated by step 1
HH = pd.read_csv(HHf).drop_duplicates()
II = pd.read_csv(IIf).drop_duplicates()
DH = pd.read_csv(DHf).drop_duplicates()
DI = pd.read_csv(DIf).drop_duplicates()

In [8]:
ADMETf = 'source_data/ingredient_ADMET_Properties.xlsx'
ADMET = pd.read_excel(ADMETf).drop_duplicates()

In [9]:
ADMET.head()

,Unnamed: 0,tcmsp_herb_id,tcmsp_ingredient_id,mol_inchikey_dict,tcmsp_herb_child_id,tcmsp_herb_cn_name,tcmsp_herb_pinyin,tcmsp_herb_en_name,child_cn_name,child_en_name,...,tcmsp_ingredient_hdon,tcmsp_ingredient_hacc,tcmsp_ingredient_drug_likeness,inchikey,tcmsp_ingredient_pubChem_Cid,tcmsp_ingredient_halflife,tcmsp_ingredient_RBN,tcmsp_ingredient_TPSA,tcmsp_ingredient_FASA,tcmsp_ingredient_isosmiles
0,0,H00001,MOL010127,AOGQPLXWSUTHQB-UHFFFAOYSA-N,26,矮地茶,Aidicha,Ardisiae Japonicae Herba,止咳平喘药,Antitussive Antiasthmetics,...,0.0,2.0,0.015933,AOGQPLXWSUTHQB-UHFFFAOYSA-N,8908,NaN,6.0,26.30,0.195410,CCCCCCOC(=O)C
1,1,H00001,MOL000108,IKGXIBQEEMLURG-GALJDQSWSA-O,26,矮地茶,Aidicha,Ardisiae Japonicae Herba,止咳平喘药,Antitussive Antiasthmetics,...,10.0,16.0,0.682430,IKGXIBQEEMLURG-GALJDQSWSA-O,Not Available,NaN,6.0,269.43,0.314577,Oc1cc(O)c2c(c1)[OH+]C(=C(C2=O)O[C@@H]1O[C@@H](...
2,2,H00001,MOL010930,PMMLIVYPEUJENN-KGLIPLIRSA-N,26,矮地茶,Aidicha,Ardisiae Japonicae Herba,止咳平喘药,Antitussive Antiasthmetics,...,0.0,0.0,0.090014,PMMLIVYPEUJENN-KGLIPLIRSA-N,Not Available,7.773161,0.0,0.00,0.258104,C=C1CCCC(=C)[C@@H]2[C@H](CC1)C(C2)(C)C
3,3,H00001,MOL010931,SZOPSAFLRCYJCX-ITEOXOHJSA-N,26,矮地茶,Aidicha,Ardisiae Japonicae Herba,止咳平喘药,Antitussive Antiasthmetics,...,4.0,7.0,0.335130,SZOPSAFLRCYJCX-ITEOXOHJSA-N,9820702,NaN,5.0,116.45,0.296441,OC[C@H]1O[C@@H](O[C@@H](/C=C/[C@H]2C(=CC(=O)CC...
4,4,H00001,MOL010932,MDCGEAGEQVMWPE-SXGMEYSMSA-N,26,矮地茶,Aidicha,Ardisiae Japonicae Herba,止咳平喘药,Antitussive Antiasthmetics,...,1.0,2.0,0.065360,MDCGEAGEQVMWPE-SXGMEYSMSA-N,13857510,7.814002,2.0,37.30,0.321478,C[C@H](/C=C/[C@H]1C(=CC(=O)CC1(C)C)C)O


In [10]:
ADMET.describe()

,Unnamed: 0,tcmsp_herb_child_id,tcmsp_ingredient_ob,tcmsp_ingredient_mw,tcmsp_ingredient_alogp,tcmsp_ingredient_caco2,tcmsp_ingredient_bbb,tcmsp_ingredient_hdon,tcmsp_ingredient_hacc,tcmsp_ingredient_drug_likeness,tcmsp_ingredient_halflife,tcmsp_ingredient_RBN,tcmsp_ingredient_TPSA,tcmsp_ingredient_FASA
count,29384.000000,29384.000000,29377.000000,29377.000000,29377.000000,29377.000000,29377.000000,29377.000000,29377.000000,29377.000000,13882.000000,29377.000000,29377.000000,29377.000000
mean,14691.500000,18.054996,31.425416,322.632589,3.279170,0.502218,0.138665,2.269292,4.465943,0.271437,8.464850,5.275215,73.348401,0.242464
std,8482.574491,10.893124,20.542820,204.650855,3.113182,1.312244,1.734851,3.128166,5.208893,0.279895,5.942219,5.915780,84.822552,0.118449
min,0.000000,1.000000,0.192405,27.030000,-15.978000,-9.023440,-17.946870,0.000000,0.000000,0.000049,-22.620130,0.000000,0.000000,0.000000
25%,7345.750000,6.000000,16.440688,176.240000,1.410000,-0.002510,-0.716850,0.000000,1.000000,0.043359,4.840590,1.000000,20.230000,0.199027
50%,14691.500000,17.000000,28.972769,278.380000,2.873000,0.894540,0.555040,1.000000,3.000000,0.133880,7.911940,3.000000,44.450000,0.252002
75%,22037.250000,26.000000,43.666663,416.810000,5.018000,1.374800,1.415840,3.000000,6.000000,0.488800,11.633100,7.000000,99.380000,0.317056
max,29383.000000,37.000000,151.523502,2102.440000,30.546000,2.530990,2.450460,30.000000,52.000000,0.951160,77.014545,86.000000,877.360000,0.890721


In [11]:
%%time 

herb_commuties_case_obj = Herb_Comb_Community(HH, II, DH, DI, disease_name)
G, communities, DHI_result, subset_data_I, subset_data_H, node_detect_df = herb_commuties_case_obj.get_herb_communities(ADMET_filter = True)

start to loop for best distance cut[0.0, 0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6000000000000001, 0.7000000000000001, 0.8, 0.9, 1.0, 1.1, 1.2000000000000002, 1.3, 1.4000000000000001, 1.5, 1.6, 1.7000000000000002, 1.8, 1.9000000000000001, 2.0, 2.1, 2.2, 2.3000000000000003, 2.4000000000000004, 2.5, 2.6, 2.7, 2.8000000000000003, 2.9000000000000004, 3.0, 3.1, 3.2, 3.3000000000000003, 3.4000000000000004, 3.5, 3.6, 3.7, 3.8000000000000003, 3.9000000000000004, 4.0, 4.1000000000000005, 4.2]


  0%|          | 0/43 [00:00<?, ?it/s]c:\Users\yin\OneDrive\Project\project_group\manuscipt-jiaqi\pytomedicine\revise_v1\final\code\code\herbSyner_Finder\get_combination_community\community_detection.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_file['distance1'] = max(sum_file['distance']) - sum_file['distance']
  2%|▏         | 1/43 [00:01<00:46,  1.12s/it]c:\Users\yin\OneDrive\Project\project_group\manuscipt-jiaqi\pytomedicine\revise_v1\final\code\code\herbSyner_Finder\get_combination_community\community_detection.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

Q value is 0.3008419053514408


c:\Users\yin\OneDrive\Project\project_group\manuscipt-jiaqi\pytomedicine\revise_v1\final\code\code\herbSyner_Finder\get_combination_community\community_detection.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_file['distance1'] = max(sum_file['distance']) - sum_file['distance']
 14%|█▍        | 6/43 [00:10<01:15,  2.04s/it]

Q value is 0.31125776911374436


...

Q value is 0.3570457834808383


c:\Users\yin\OneDrive\Project\project_group\manuscipt-jiaqi\pytomedicine\revise_v1\final\code\code\herbSyner_Finder\get_combination_community\community_detection.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_file['distance1'] = max(sum_file['distance']) - sum_file['distance']
 33%|███▎      | 14/43 [00:30<01:06,  2.28s/it]

Q value is 0.362469598834008


c:\Users\yin\OneDrive\Project\project_group\manuscipt-jiaqi\pytomedicine\revise_v1\final\code\code\herbSyner_Finder\get_combination_community\community_detection.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_file['distance1'] = max(sum_file['distance']) - sum_file['distance']
 35%|███▍      | 15/43 [00:32<01:01,  2.18s/it]c:\Users\yin\OneDrive\Project\project_group\manuscipt-jiaqi\pytomedicine\revise_v1\final\code\code\herbSyner_Finder\get_combination_community\community_detection.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

Loop the distance from 0.0 to 4.2 with max Q = 0.362469598834008 at distance cut of 1.3
Node Cough Variant Asthma is at index: 184
The element 184 is in the 4 set.
CPU times: total: 10min 9s
Wall time: 10min 11s


In [12]:
subset_data_I

,Disease name,Herb ID,Herb name,Ingredient ID,Ingredient name,distance,Z-score,Mean-value,stand_deviation,p-value,tcmsp_ingredient_name,tcmsp_ingredient_ob,tcmsp_ingredient_drug_likeness
21,Cough Variant Asthma,H3794,MA HUANG,I135,luteolin,2.195767,0.744978,2.158201,0.050426,"(0.78,)",luteolin,36.162629,0.24552
309,Cough Variant Asthma,H3794,MA HUANG,I28341,(+)-Medicarpin,3.307692,1.311468,2.849396,0.349453,"(0.92,)",(+)-Medicarpin,60.457474,0.33505
596,Cough Variant Asthma,H5211,SHENG JIANG,I14021,naringenin,2.562162,1.968974,2.414378,0.075056,"(0.98,)",naringenin,59.293898,0.21128
710,Cough Variant Asthma,H7468,ZI WAN,I135,luteolin,2.195767,0.744978,2.158201,0.050426,"(0.78,)",luteolin,36.162629,0.24552
1242,Cough Variant Asthma,H1210,DA ZAO,I236,beta-carotene,2.765027,2.661411,2.459180,0.114919,"(0.96,)",beta-carotene,37.184333,0.58358
1411,Cough Variant Asthma,H1210,DA ZAO,I751,ellagic acid,2.766304,0.866724,2.567772,0.229061,"(0.82,)",ellagic acid,43.064559,0.43417
1433,Cough Variant Asthma,H1210,DA ZAO,I7111,berberine,2.226316,1.043554,2.173421,0.050687,"(0.85,)",berberine,36.861245,0.77665
1691,Cough Variant Asthma,H343,BAN XIA,I518,Baicalin,2.917582,2.096059,2.538407,0.180899,"(0.94,)",Baicalin,40.123610,0.75264
1818,Cough Variant Asthma,H5972,WU WEI ZI,I151,podophyllotoxin,2.500000,2.833038,2.312228,0.066279,"(1.0,)",podophyllotoxin,59.937690,0.85670


# 3. Further detect the key module with main synergistic effects

In [13]:
# prepare necessary files
comb_data_ingre_list = [subset_data_I]
comb_name_ingre_list = ['SH']
comb_data_herb_list = [subset_data_H]
comb_name_herb_list = ['SH']
comb_data_merged_list = [herb_commuties_case_obj.sum_file_all]
comb_name_merged_list = ['SH']
mocule_dataset_list = [comb_data_ingre_list, 
                       comb_name_ingre_list,
                       comb_data_herb_list,
                        comb_name_herb_list,
                        comb_data_merged_list,
                        comb_name_merged_list]

In [14]:
herb_commuties_case_obj.sum_file_all

,node_from,node_to,distance
0,Acetovanillone,Tetradecanoic acid,2.142857
1,Acetovanillone,PENTADECANOIC ACID,2.672131
2,Acetovanillone,HEXANOIC ACID,2.645161
3,Acetovanillone,lauric acid,2.552239
4,Acetovanillone,kaempferol,1.928571
...,...,...,...
31,KUAN DONG HUA,BAN XIA,0.984709
32,KUAN DONG HUA,WU WEI ZI,1.521690
33,DA ZAO,BAN XIA,0.752623
34,DA ZAO,WU WEI ZI,1.512809


In [15]:
%%time 
# Detection_key_Module_multiple
module_multiple = Detection_key_Module_multiple(*mocule_dataset_list)
module_multiple.prepare_downstream_data(save_path)

CPU times: total: 62.5 ms
Wall time: 84.4 ms


c:\Users\yin\OneDrive\Project\project_group\manuscipt-jiaqi\pytomedicine\revise_v1\final\code\code\herbSyner_Finder\get_combination_community\community_detection.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comb_one['FJ'] = comb_name_list[i]
